In [ ]:
import torch
import torch.nn as nn
import torchvision.models as model


class VGG16(nn.Module):
    
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
                
        self.vgg16 = model.vgg16(pretrained=pretrained)
        self.vgg16.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, image):
        out = self.vgg16(image)
        
        return out


In [3]:
import torch
import torch.nn as nn
from torch.nn.functional import softmax
from torchmetrics import F1Score
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18, vgg16
from torchvision.datasets import ImageFolder
from torch.optim import SGD, Adam
from torch.utils.tensorboard import SummaryWriter
import tqdm
import os

save_model_path = "checkpoints/"


def val(model, data_val, loss_function, writer, epoch, device):
    f1score = 0
    f1 = F1Score(num_classes=47, task='multiclass')
    data_iterator = enumerate(data_val)  # Take batches
    f1_list = []
    f1t_list = []

    with torch.no_grad():
        model.eval()  # Switch model to evaluation mode
        tq = tqdm.tqdm(total=len(data_val))
        tq.set_description('Validation:')

        total_loss = 0

        for _, batch in data_iterator:
            # Forward propagation
            image, label = batch
            image = image.to(device)
            label = label.to(device).long()  # Convert labels to LongTensor
            
            pred = model(image)

            loss = loss_function(pred, label)

            pred = pred.softmax(dim=1)
            
            f1_list.extend(torch.argmax(pred, dim=1).tolist())
            f1t_list.extend(label.tolist())  # Labels are already in LongTensor format

            total_loss += loss.item()
            tq.update(1)

    f1score = f1(torch.tensor(f1_list), torch.tensor(f1t_list))
    writer.add_scalar("Validation F1", f1score, epoch)
    writer.add_scalar("Validation Loss", total_loss / len(data_val), epoch)

    tq.close()
    print("F1 score: ", f1score)

    return None


def train(model, train_loader, val_loader, optimizer, loss_fn, n_epochs, device, log_dir):
    writer = SummaryWriter(log_dir=os.path.join('runs', log_dir))

    model.to(device)  # Move the model to the specified device (e.g., GPU or CPU)
    model.train()  # Set the model to training mode

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0

        tq = tqdm.tqdm(total=len(train_loader))
        tq.set_description('epoch %d' % epoch)

        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)  # Move the batch of images to the specified device
            labels = labels.to(device).long()  # Convert labels to LongTensor
            
            optimizer.zero_grad()  # Reset the gradients of the optimizer

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = loss_fn(outputs, labels)
            outputs = outputs.softmax(dim=1)

            # Backward pass
            loss.backward()

            # Update model parameters
            optimizer.step()

            running_loss += loss.item()
            tq.set_postfix(loss_st='%.6f' % loss.item())
            tq.update(1)

        writer.add_scalar("Training Loss", running_loss / len(train_loader), epoch)

        tq.close()
        epoch_loss = running_loss / len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, n_epochs, epoch_loss))

        # Check the performance of the model on the validation dataset
        val(model, val_loader, loss_fn, writer, epoch, device)

        # Save the model in .pth format
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        torch.save(checkpoint, os.path.join(save_model_path, log_dir + '.pth'))
        print("Saved the model to " + save_model_path)


def main(model_name, optimizer_name, lr, pretrained, log_dir):
    device = "cuda"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    # Data transformations
    tr_train = transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(30),
        transforms.Resize([250, 250]),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
    ])
    tr_val = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
    ])

    # Dataset and DataLoader
    train_data = ImageFolder(root=r"datasets\split_data\train", transform=tr_train)
    val_data = ImageFolder(root=r"datasets\split_data\val", transform=tr_val)

    train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=8, drop_last=True)

    max_epoch = 20

    # Model selection and freezing logic
    if model_name == 'resnet':
        model = resnet18(pretrained=pretrained).to(device)
        if pretrained:
            # Freeze all layers except the final layer
            for param in model.parameters():
                param.requires_grad = False
            # Replace the fully connected layer
            model.fc = nn.Linear(model.fc.in_features, 4)
        else:
            # Directly modify the FC layer
            model.fc = nn.Linear(model.fc.in_features, 4)

    elif model_name == 'vgg':
        model = vgg16(pretrained=pretrained).to(device)
        if pretrained:
            # Freeze all layers except the classifier
            for param in model.features.parameters():
                param.requires_grad = False
            # Replace the classifier with a new one
            model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 4)
        else:
            # Directly modify the classifier
            model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 4)

    # Optimizer selection
    if optimizer_name == 'sgd':
        optimizer = SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9)
    elif optimizer_name == 'adam':
        optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    # Loss function
    loss = nn.CrossEntropyLoss()

    # Start training
    train(model, train_loader, val_loader, optimizer, loss, max_epoch, device, log_dir)




In [4]:
if __name__ == "__main__":
    
    # main(model_name='resnet', optimizer_name='sgd', lr=1e-2, pretrained=False, log_dir='resnet_sgd')
    # main(model_name='resnet', optimizer_name='adam', lr=1e-4, pretrained=False, log_dir='resnet_adam')
    
    main(model_name='resnet', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='freeze_resnet_sgd_pretrained')
    main(model_name='resnet', optimizer_name='adam', lr=1e-4, pretrained=True, log_dir='freeze_resnet_adam_pretrained')
    
    # main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=False, log_dir='vgg_sgd')
    # main(model_name='vgg', optimizer_name='adam', lr=1e-4, pretrained=False, log_dir='vgg_adam')
    
    # main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='freeze_vgg_sgd_pretrained')
    # main(model_name='vgg', optimizer_name='adam', lr=1e-4, pretrained=True, log_dir='freeze_vgg_adam_pretrained')

epoch 0: 100%|██████████| 350/350 [00:12<00:00, 28.83it/s, loss_st=1.551793]


Epoch [1/20], Loss: 1.1809


Validation:: 100%|██████████| 50/50 [00:01<00:00, 36.26it/s]


F1 score:  tensor(0.6175)
Saved the model to checkpoints/


epoch 1: 100%|██████████| 350/350 [00:11<00:00, 30.81it/s, loss_st=1.259058]


...

F1 score:  tensor(0.7175)
Saved the model to checkpoints/


epoch 19: 100%|██████████| 350/350 [00:11<00:00, 30.83it/s, loss_st=0.891731]


Epoch [20/20], Loss: 0.9378


Validation:: 100%|██████████| 50/50 [00:01<00:00, 37.92it/s]

F1 score:  tensor(0.7175)
Saved the model to checkpoints/


In [5]:
if __name__ == "__main__":
    
    main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='freeze_vgg_sgd_pretrained')
    main(model_name='vgg', optimizer_name='adam', lr=1e-5, pretrained=True, log_dir='freeze_vgg_adam_pretrained')

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
epoch 0: 100%|██████████| 350/350 [00:22<00:00, 15.31it/s, loss_st=0.739610]


Epoch [1/20], Loss: 1.1455


Validation:: 100%|██████████| 50/50 [00:02<00:00, 23.76it/s]


F1 score:  tensor(0.6600)
Saved the model to checkpoints/


epoch 1: 100%|██████████| 350/350 [00:22<00:00, 15.37it/s, loss_st=0.786201]


...

F1 score:  tensor(0.7575)
Saved the model to checkpoints/


epoch 19: 100%|██████████| 350/350 [00:30<00:00, 11.33it/s, loss_st=0.241274]


Epoch [20/20], Loss: 0.2080


Validation:: 100%|██████████| 50/50 [00:02<00:00, 23.63it/s]


F1 score:  tensor(0.7375)
Saved the model to checkpoints/


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as model
from torchmetrics import F1Score
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
import tqdm

# Define the VGG16 model class
class VGG16(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        self.vgg16 = model.vgg16(pretrained=pretrained)
        self.vgg16.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, image):
        out = self.vgg16(image)
        return out

# Function to evaluate accuracy on the test dataset with classwise accuracy
def test(model, test_loader, device, num_classes):
    correct_preds = torch.zeros(num_classes).to(device)  # Correct predictions per class
    total_preds = torch.zeros(num_classes).to(device)  # Total predictions per class
    model.eval()  # Switch model to evaluation mode

    with torch.no_grad():
        for images, labels in tqdm.tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class

            # Update counts for each class
            for i in range(num_classes):
                correct_preds[i] += ((predicted == i) & (labels == i)).sum().item()  # Correct predictions for class i
                total_preds[i] += (labels == i).sum().item()  # Total samples for class i

    # Calculate accuracy per class
    classwise_accuracy = correct_preds / total_preds * 100

    # Print the results
    for i in range(num_classes):
        print(f"Class {i} Accuracy: {classwise_accuracy[i]:.2f}%")

    return classwise_accuracy

# Function to load model from checkpoint
def load_model(checkpoint_path, model, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    model.eval()
    print(f"Model loaded successfully from {checkpoint_path}")
    return model

# Function to prepare the test dataset
def prepare_test_data(test_data_path, batch_size):
    tr_test = transforms.Compose([
        transforms.Resize([224, 224]),  # Resize images
        transforms.ToTensor(),  # Convert to tensor
    ])
    
    # Load the test dataset
    test_data = ImageFolder(root=test_data_path, transform=tr_test)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)  # No need to shuffle for testing
    print(f"Test dataset size: {len(test_data)} samples")
    return test_loader

# Main function to train, evaluate, and load models
def main(model_name, optimizer_name, lr, pretrained, log_dir, test_data_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    if model_name == 'vgg':
        model = VGG16(num_classes=4, pretrained=pretrained).to(device)
    elif model_name == 'resnet':
        model = ResNet18(num_classes=4, pretrained=pretrained).to(device)

    # Load the model
    checkpoint_path = os.path.join("checkpoints", log_dir + '.pth')
    model = load_model(checkpoint_path, model, device)

    # Prepare the test data
    test_loader = prepare_test_data(test_data_path, batch_size=8)

    # Evaluate the model on the test data
    num_classes = 4  # Set the number of classes in your dataset
    classwise_accuracy = test(model, test_loader, device, num_classes)
    print(f"Classwise Accuracy for {model_name} ({log_dir}): {classwise_accuracy}")

if __name__ == "__main__":
    test_data_path = r"datasets\split_data\test"  # Path to your test dataset

    # Example: Test with a pretrained VGG model trained with SGD
    main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='vgg_sgd_pretrained', test_data_path=test_data_path)
    main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='vgg_adam_pretrained', test_data_path=test_data_path)


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11788\989400935.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See 

Model loaded successfully from checkpoints\vgg_sgd_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:11<00:00,  9.12it/s]


Class 0 Accuracy: 88.06%
Class 1 Accuracy: 88.56%
Class 2 Accuracy: 79.60%
Class 3 Accuracy: 80.60%
Classwise Accuracy for vgg (vgg_sgd_pretrained): tensor([88.0597, 88.5572, 79.6020, 80.5970], device='cuda:0')
Model loaded successfully from checkpoints\vgg_adam_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 21.24it/s]

Class 0 Accuracy: 76.12%
Class 1 Accuracy: 90.05%
Class 2 Accuracy: 92.04%
Class 3 Accuracy: 60.70%
Classwise Accuracy for vgg (vgg_adam_pretrained): tensor([76.1194, 90.0498, 92.0398, 60.6965], device='cuda:0')


In [3]:
import torch
import torch.nn as nn
import torchvision.models as model
from torchmetrics import F1Score
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
import tqdm

# Define the VGG16 model class
class VGG16(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        self.vgg16 = model.vgg16(pretrained=pretrained)
        self.vgg16.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, image):
        out = self.vgg16(image)
        return out
    # Define the ResNet model class
class ResNet18(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        self.resnet18 = model.resnet18(pretrained=pretrained)
        self.resnet18.fc = torch.nn.Linear(in_features=self.resnet18.fc.in_features, out_features=num_classes)

    def forward(self, image):
        out = self.resnet18(image)
        return out
    def load_state_dict(self, state_dict, strict=True):
        """
        Custom load_state_dict to handle key mismatches by adding 'resnet18.' prefix.
        """
        # Add 'resnet18.' prefix to each key in the state_dict
        state_dict = {f"resnet18.{key}": value for key, value in state_dict.items()}
        # Now load the modified state_dict into the model
        super().load_state_dict(state_dict, strict)


# Function to evaluate accuracy on the test dataset with classwise accuracy
def test(model, test_loader, device, num_classes):
    correct_preds = torch.zeros(num_classes).to(device)  # Correct predictions per class
    total_preds = torch.zeros(num_classes).to(device)  # Total predictions per class
    model.eval()  # Switch model to evaluation mode

    with torch.no_grad():
        for images, labels in tqdm.tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class

            # Update counts for each class
            for i in range(num_classes):
                correct_preds[i] += ((predicted == i) & (labels == i)).sum().item()  # Correct predictions for class i
                total_preds[i] += (labels == i).sum().item()  # Total samples for class i

    # Calculate accuracy per class
    classwise_accuracy = correct_preds / total_preds * 100

    # Print the results
    for i in range(num_classes):
        print(f"Class {i} Accuracy: {classwise_accuracy[i]:.2f}%")

    return classwise_accuracy

# Function to load model from checkpoint
def load_model(checkpoint_path, model, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'], strict=False)

    model.to(device)
    model.eval()
    print(f"Model loaded successfully from {checkpoint_path}")
    return model

# Function to prepare the test dataset
def prepare_test_data(test_data_path, batch_size):
    tr_test = transforms.Compose([
        transforms.Resize([224, 224]),  # Resize images
        transforms.ToTensor(),  # Convert to tensor
    ])
    
    # Load the test dataset
    test_data = ImageFolder(root=test_data_path, transform=tr_test)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)  # No need to shuffle for testing
    print(f"Test dataset size: {len(test_data)} samples")
    return test_loader

# Main function to train, evaluate, and load models
def main(model_name, optimizer_name, lr, pretrained, log_dir, test_data_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    if model_name == 'vgg':
        model = VGG16(num_classes=4, pretrained=pretrained).to(device)
    elif model_name == 'resnet':
        model = ResNet18(num_classes=4, pretrained=pretrained).to(device)

    # Load the model
    checkpoint_path = os.path.join("checkpoints", log_dir + '.pth')
    model = load_model(checkpoint_path, model, device)

    # Prepare the test data
    test_loader = prepare_test_data(test_data_path, batch_size=8)

    # Evaluate the model on the test data
    num_classes = 4  # Set the number of classes in your dataset
    classwise_accuracy = test(model, test_loader, device, num_classes)
    print(f"Classwise Accuracy for {model_name} ({log_dir}): {classwise_accuracy}")




In [6]:
test_data_path = r"datasets\split_data\test"  # Path to your test dataset
main(model_name='resnet', optimizer_name='sgd', lr=1e-3, pretrained=False, log_dir='resnet_sgd', test_data_path=test_data_path)
main(model_name='resnet', optimizer_name='adam', lr=1e-5, pretrained=False, log_dir='resnet_adam', test_data_path=test_data_path)

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14952\186437167.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future rel

Model loaded successfully from checkpoints\resnet_sgd.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:06<00:00, 16.31it/s]


Class 0 Accuracy: 24.88%
Class 1 Accuracy: 64.18%
Class 2 Accuracy: 74.63%
Class 3 Accuracy: 56.22%
Classwise Accuracy for resnet (resnet_sgd): tensor([24.8756, 64.1791, 74.6269, 56.2189], device='cuda:0')
Model loaded successfully from checkpoints\resnet_adam.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:02<00:00, 49.57it/s]

Class 0 Accuracy: 78.11%
Class 1 Accuracy: 52.74%
Class 2 Accuracy: 68.16%
Class 3 Accuracy: 66.67%
Classwise Accuracy for resnet (resnet_adam): tensor([78.1095, 52.7363, 68.1592, 66.6667], device='cuda:0')


In [7]:
test_data_path = r"datasets\split_data\test"  # Path to your test dataset
main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=False, log_dir='vgg_sgd', test_data_path=test_data_path)
main(model_name='vgg', optimizer_name='adam', lr=1e-5, pretrained=False, log_dir='vgg_adam', test_data_path=test_data_path)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14952\186437167.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)

Model loaded successfully from checkpoints\vgg_sgd.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 23.95it/s]


Class 0 Accuracy: 12.44%
Class 1 Accuracy: 1.00%
Class 2 Accuracy: 6.97%
Class 3 Accuracy: 97.01%
Classwise Accuracy for vgg (vgg_sgd): tensor([12.4378,  0.9950,  6.9652, 97.0149], device='cuda:0')
Model loaded successfully from checkpoints\vgg_adam.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 23.47it/s]

Class 0 Accuracy: 0.00%
Class 1 Accuracy: 0.00%
Class 2 Accuracy: 0.00%
Class 3 Accuracy: 100.00%
Classwise Accuracy for vgg (vgg_adam): tensor([  0.,   0.,   0., 100.], device='cuda:0')


In [10]:
test_data_path = r"datasets\split_data\test"  # Path to your test dataset
main(model_name='resnet', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='freeze_resnet_sgd_pretrained', test_data_path=test_data_path)
main(model_name='resnet', optimizer_name='adam', lr=1e-5, pretrained=True, log_dir='freeze_resnet_adam_pretrained', test_data_path=test_data_path)

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14952\186437167.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling

Model loaded successfully from checkpoints\freeze_resnet_sgd_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:02<00:00, 44.23it/s]


Class 0 Accuracy: 40.80%
Class 1 Accuracy: 70.65%
Class 2 Accuracy: 75.12%
Class 3 Accuracy: 60.70%
Classwise Accuracy for resnet (freeze_resnet_sgd_pretrained): tensor([40.7960, 70.6468, 75.1244, 60.6965], device='cuda:0')
Model loaded successfully from checkpoints\freeze_resnet_adam_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:02<00:00, 42.56it/s]

Class 0 Accuracy: 62.69%
Class 1 Accuracy: 65.17%
Class 2 Accuracy: 65.67%
Class 3 Accuracy: 55.22%
Classwise Accuracy for resnet (freeze_resnet_adam_pretrained): tensor([62.6866, 65.1741, 65.6716, 55.2239], device='cuda:0')


In [14]:
from torchvision.models import resnet18, vgg16
def main(model_name, optimizer_name, lr, pretrained, log_dir, test_data_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    if model_name == 'resnet':
        model = resnet18(pretrained=pretrained).to(device)
        if pretrained:
            # Freeze all layers except the final layer
            for param in model.parameters():
                param.requires_grad = False
            # Replace the fully connected layer
            model.fc = nn.Linear(model.fc.in_features, 4)
        else:
            # Directly modify the FC layer
            model.fc = nn.Linear(model.fc.in_features, 4)

    elif model_name == 'vgg':
        model = vgg16(pretrained=pretrained).to(device)
        if pretrained:
            # Freeze all layers except the classifier
            for param in model.features.parameters():
                param.requires_grad = False
            # Replace the classifier with a new one
            model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 4)
        else:
            # Directly modify the classifier
            model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 4)

    # Load the model
    checkpoint_path = os.path.join("checkpoints", log_dir + '.pth')
    model = load_model(checkpoint_path, model, device)

    # Prepare the test data
    test_loader = prepare_test_data(test_data_path, batch_size=8)

    # Evaluate the model on the test data
    num_classes = 4  # Set the number of classes in your dataset
    classwise_accuracy = test(model, test_loader, device, num_classes)
    print(f"Classwise Accuracy for {model_name} ({log_dir}): {classwise_accuracy}")

    

In [15]:
test_data_path = r"datasets\split_data\test"  # Path to your test dataset
main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='freeze_vgg_sgd_pretrained', test_data_path=test_data_path)
main(model_name='vgg', optimizer_name='adam', lr=1e-5, pretrained=True, log_dir='freeze_vgg_adam_pretrained', test_data_path=test_data_path)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14952\186437167.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)

Model loaded successfully from checkpoints\freeze_vgg_sgd_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 23.96it/s]


Class 0 Accuracy: 73.63%
Class 1 Accuracy: 56.72%
Class 2 Accuracy: 69.15%
Class 3 Accuracy: 79.60%
Classwise Accuracy for vgg (freeze_vgg_sgd_pretrained): tensor([73.6318, 56.7164, 69.1542, 79.6020], device='cuda:0')
Model loaded successfully from checkpoints\freeze_vgg_adam_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 23.68it/s]

Class 0 Accuracy: 76.62%
Class 1 Accuracy: 60.70%
Class 2 Accuracy: 77.11%
Class 3 Accuracy: 75.62%
Classwise Accuracy for vgg (freeze_vgg_adam_pretrained): tensor([76.6169, 60.6965, 77.1144, 75.6219], device='cuda:0')


In [18]:
import torch
import torch.nn as nn
import torchvision.models as model
from torchmetrics import F1Score
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
import tqdm


# Function to evaluate accuracy on the test dataset
def test(model, test_loader, device):
    correct_preds = 0
    total_preds = 0
    model.eval()  # Switch model to evaluation mode

    with torch.no_grad():
        for images, labels in tqdm.tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class

            # Update counts
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

    accuracy = correct_preds / total_preds
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy

# Function to load model from checkpoint
def load_model(checkpoint_path, model, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    model.eval()
    print(f"Model loaded successfully from {checkpoint_path}")
    return model

# Function to prepare the test dataset
def prepare_test_data(test_data_path, batch_size):
    tr_test = transforms.Compose([
        transforms.Resize([224, 224]),  # Resize images
        transforms.ToTensor(),  # Convert to tensor
    ])
    
    # Load the test dataset
    test_data = ImageFolder(root=test_data_path, transform=tr_test)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)  # No need to shuffle for testing
    print(f"Test dataset size: {len(test_data)} samples")
    return test_loader

def main(model_name, optimizer_name, lr, pretrained, log_dir, test_data_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    if model_name == 'resnet':
        model = resnet18(pretrained=pretrained).to(device)
        if pretrained:
            # Freeze all layers except the final layer
            for param in model.parameters():
                param.requires_grad = False
            # Replace the fully connected layer
            model.fc = nn.Linear(model.fc.in_features, 4)
        else:
            # Directly modify the FC layer
            model.fc = nn.Linear(model.fc.in_features, 4)

    elif model_name == 'vgg':
        model = vgg16(pretrained=pretrained).to(device)
        if pretrained:
            # Freeze all layers except the classifier
            for param in model.features.parameters():
                param.requires_grad = False
            # Replace the classifier with a new one
            model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 4)
        else:
            # Directly modify the classifier
            model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 4)


    # Load the model
    checkpoint_path = os.path.join("checkpoints", log_dir + '.pth')
    model = load_model(checkpoint_path, model, device)

    # Prepare the test data
    test_loader = prepare_test_data(test_data_path, batch_size=8)

    # Evaluate the model on the test data
    test_accuracy = test(model, test_loader, device)
    print(f"Test Accuracy for {model_name} ({log_dir}): {test_accuracy:.4f}")

if __name__ == "__main__":
    test_data_path = r"datasets\split_data\test"  # Path to your test dataset

    # Example: Test with a pretrained VGG model trained with SGD
    main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='freeze_vgg_sgd_pretrained', test_data_path=test_data_path)
    main(model_name='vgg', optimizer_name='adam', lr=1e-5, pretrained=True, log_dir='freeze_vgg_adam_pretrained', test_data_path=test_data_path)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14952\1996011608.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device

Model loaded successfully from checkpoints\freeze_vgg_sgd_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 23.88it/s]


Test Accuracy: 69.78%
Test Accuracy for vgg (freeze_vgg_sgd_pretrained): 0.6978
Model loaded successfully from checkpoints\freeze_vgg_adam_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 24.15it/s]

Test Accuracy: 72.51%
Test Accuracy for vgg (freeze_vgg_adam_pretrained): 0.7251


In [19]:
test_data_path = r"datasets\split_data\test"  # Path to your test dataset
main(model_name='resnet', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='freeze_resnet_sgd_pretrained', test_data_path=test_data_path)
main(model_name='resnet', optimizer_name='adam', lr=1e-5, pretrained=True, log_dir='freeze_resnet_adam_pretrained', test_data_path=test_data_path)

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14952\1996011608.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling.

Model loaded successfully from checkpoints\freeze_resnet_sgd_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:02<00:00, 37.71it/s]


Test Accuracy: 61.82%
Test Accuracy for resnet (freeze_resnet_sgd_pretrained): 0.6182
Model loaded successfully from checkpoints\freeze_resnet_adam_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:02<00:00, 39.18it/s]

Test Accuracy: 62.19%
Test Accuracy for resnet (freeze_resnet_adam_pretrained): 0.6219
